# Bike Supply

## Summary

Bike Supply is a PyMC3-driven Bayesian predictive model of an urban bike share company's supply-demand dynamics. This model was built using public data supplied by Bay Area Bike Share. This model was built to answer the question - is it possible to build a model that predicts the volume of bikes entering and leaving a station for any given day of the year?

Bike sharing programs have the unique challenge of needing to ensure bikes and docking stations are available throughout the city for users throughout the day, even though users have the freedom to choose their own destinations rental by rental. 

## About the dataset

## Data prep

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint
%matplotlib inline

data = pd.read_csv('bikeshare_trips.csv')

In [30]:
data[0:3]

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type,year,month,start_day,start_time,end_day,end_time,trip_length
0,650330,495,2015-02-19 15:46:00,Yerba Buena Center of the Arts (3rd @ Howard),68,2015-02-19 15:54:00,San Francisco Caltrain 2 (330 Townsend),69,463,94061,Subscriber,2015,2,19,15:46:00,19,15:54:00,00:08:00
1,367544,241,2014-07-17 08:09:00,San Francisco Caltrain 2 (330 Townsend),69,2014-07-17 08:13:00,Townsend at 7th,65,441,95120,Subscriber,2014,7,17,08:09:00,17,08:13:00,00:04:00
2,586344,3589,2014-12-27 16:01:00,Embarcadero at Bryant,54,2014-12-27 17:01:00,2nd at Townsend,61,466,27401,Customer,2014,12,27,16:01:00,27,17:01:00,01:00:00


In [29]:
print "Number of stations:", len(data.start_station_name.unique())
print "Number of bikes:", len(data.bike_number.unique())
print "Number of trips logged:", len(data)

Number of stations: 84
Number of bikes: 700
Number of trips logged: 983648


In [7]:
# Remove UTC from datatime !!Don't do this more than once!!
data['start_date'] = data['start_date'].astype(str).str[:-3]

In [25]:
data['start_date'] = pd.to_datetime(data['start_date'])

In [9]:
data['end_date'] = pd.to_datetime(data['end_date'])

In [31]:
data[0:3]

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type,year,month,start_day,start_time,end_day,end_time,trip_length
0,650330,495,2015-02-19 15:46:00,Yerba Buena Center of the Arts (3rd @ Howard),68,2015-02-19 15:54:00,San Francisco Caltrain 2 (330 Townsend),69,463,94061,Subscriber,2015,2,19,15:46:00,19,15:54:00,00:08:00
1,367544,241,2014-07-17 08:09:00,San Francisco Caltrain 2 (330 Townsend),69,2014-07-17 08:13:00,Townsend at 7th,65,441,95120,Subscriber,2014,7,17,08:09:00,17,08:13:00,00:04:00
2,586344,3589,2014-12-27 16:01:00,Embarcadero at Bryant,54,2014-12-27 17:01:00,2nd at Townsend,61,466,27401,Customer,2014,12,27,16:01:00,27,17:01:00,01:00:00


In [11]:
print "Number of years in dataset:", len(data.start_date.dt.year.unique())

4

In [13]:
data['year'] = data.start_date.dt.year

In [14]:
data['month'] = data.start_date.dt.month

In [15]:
data['start_day'] = data.start_date.dt.day

In [16]:
data['start_time'] = data.start_date.dt.time

In [17]:
data['end_day'] = data.end_date.dt.day

In [18]:
data['end_time'] = data.end_date.dt.time

In [19]:
data[0:5]

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type,year,month,start_day,start_time,end_day,end_time
0,650330,495,2015-02-19 15:46:00,Yerba Buena Center of the Arts (3rd @ Howard),68,2015-02-19 15:54:00,San Francisco Caltrain 2 (330 Townsend),69,463,94061,Subscriber,2015,2,19,15:46:00,19,15:54:00
1,367544,241,2014-07-17 08:09:00,San Francisco Caltrain 2 (330 Townsend),69,2014-07-17 08:13:00,Townsend at 7th,65,441,95120,Subscriber,2014,7,17,08:09:00,17,08:13:00
2,586344,3589,2014-12-27 16:01:00,Embarcadero at Bryant,54,2014-12-27 17:01:00,2nd at Townsend,61,466,27401,Customer,2014,12,27,16:01:00,27,17:01:00
3,772874,733,2015-05-19 09:50:00,Davis at Jackson,42,2015-05-19 10:02:00,2nd at Townsend,61,501,94111,Subscriber,2015,5,19,09:50:00,19,10:02:00
4,419431,302,2014-08-21 19:11:00,5th at Howard,57,2014-08-21 19:16:00,San Francisco Caltrain (Townsend at 4th),70,392,95110,Subscriber,2014,8,21,19:11:00,21,19:16:00


In [20]:
data['trip_length'] = data['end_date'] - data['start_date']

In [21]:
data['trip_length'][0:10]

0   00:08:00
1   00:04:00
2   01:00:00
3   00:12:00
4   00:05:00
5   00:05:00
6   00:20:00
7   00:06:00
8   00:15:00
9   00:07:00
Name: trip_length, dtype: timedelta64[ns]

In [22]:
max(data['trip_length'])

Timedelta('199 days 22:19:00')

In [23]:
# Remove trips longer than 3 days 




Timedelta('0 days 00:16:58.983782')

## EDA

The big question I have on the onset of this research is how granular do we need to get in our model?

* Are the dyanmics similar enough across stations such that they can all be included in the same model, or do we need to build models for clusters of stations? We will try some logical groupings to see if there are any global patterns of dynamics across them. 

* Weekday-Weekend? Day by day? Month by month? Seasonality? Let's do some side-by-side visualizations of these time periods to see if any trends emerge. 

* How granular should we be in time bins (1 hour bins, 10 minutes, 1 minute)? I think some simple histograms can help us identity the kinds of volume of activity we are seeing. 

I'm curious about the number of bikes in the fleet, month by month, year by year. I think his may map onto the growth of the company, and that there may be different dynamics at each scale of growth. This is a big question I have with this data going forward - how will this prediction model scale? I'd love to work closely with the company to observe such shifts. 


## Simple model (unaware) - historical data

Inspired by our Python maxim of KISS, let's begin by building a simple model that predicts station inputs and outputs based on hourly averages for each station. To help us out a little, we'll follow our intuition and segment weekend from weekday data, creating two spearate predictors. This first model will be network unaware, meaning we will look only at historical rentals and returns for each station, and not adjust our predictions based on what we know is happening at other stations in real time. 

For our averages we will use annual averages from 2015, and plot these alongside 2016 data points to see if our averages serve as accurate predictors. 

## Simple model (aware) - historical + "real-time" data

This model makes predictions based on the behavior of other stations in the network in real time, and takes a step closer to our Bayesian model. Still relying on annual averages from 2015 to predict departures, this model builds up a prediction of arrivals based on the sum of a weighted factor that is a product of the distribution of journey destinations (by hour) for each station, the number of bikes leaving those station (at each hour interval), and the average bike trip length for that hour interval (which will determine when the bike is predicted to arrive at its destination). NOTE: I put "real-time" in quotes, because this model is still being simulated with historical data. However, the attempt with this model is to simulate what could be done in real time.

## Bayesian model (2015 priors)

We will get a bit more granular in our setting of priors - taking some of the seasonality and day of the week trends we spotted in our EDA. Our hyperparameters of the prior distribution will be set on 2015 data, with our tau coming from the granularity we are choosing (day by day, season by season, etc.). We assume our bike rentals are Poisson-distributed. 

The goal is predict lambdas of net traffic (arrivals - departures) for each station. In the end, we will have generated 84 lambdas, illustrating the dynamics we expect to see across our 84 stations, which we will plot against our 2016 data. 


## Bayesian model (live prior updating)

This is the model I am for ....

Each bike that leaves a station updates the prior probabilities for the model such that what we are seeing in our posterior distributions are based on the actual dynamics of that day. What is that called? Is that still a Bayesian model?

## General provisions

I'm not sure how to set a good prior, but I think 2015 data to predict 2016 seems good. I may want to try 2014 + 2015, but I think it all depends on what we observe in the EDA. 

* Adjust for Giants games (some stations).

* Adjust for weather. 

Distribution of precisions across 84 stations - and an average. 

Overall precision rating in reducing errors between prediction and observed - each model gets one score (we compare those scores). 

## References